In [ ]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

import glob, io, json, os, re, requests, socket, subprocess
import pandas as pd

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
notebook_wide_display()

In [ ]:
# Read UPS names and serial numbers from this sheet
googleSheetUrl = 'https://docs.google.com/spreadsheets/d/1Klk0muDZlaXZkiyBQ8cIo9bMA6UESIPlLmFzHxszgl0/edit#gid=0'

In [ ]:
Stat.set_service('UPS')

In [ ]:
# Get serial numbers for APC devices

def findSerialNumbers():
    serialNumbers = []

    for line in subprocess_check("lsusb -v -d 051d: | grep iSerial").split('\n'):
        tokens = line.split()
        if len(tokens) == 3:
            serialNumbers.append(tokens[2])

    serialNumbers = sorted(serialNumbers)

    print('Found %d APC UPSs connected by USB, serial numbers %s' % (len(serialNumbers), serialNumbers))
    return serialNumbers


In [ ]:
def restartNut(serialNumbers):
    # Kill existing processes
    subprocess_check('killall upsd usbhid-ups', ignore_error=True)
    print('Creating /etc/nut/nut.conf')
    open('/etc/nut/nut.conf', 'w').write('# Created by upsstat\nMODE=standalone\n')
    print('Creating /etc/nut/ups.conf')
    with open('/etc/nut/ups.conf', 'w') as ups_conf:
        ups_conf.write('# Created by upsstat\n')
        ups_conf.write('maxretry = 3\n')
        for i, serialNumber in enumerate(serialNumbers):
            print('Device %d: serial=%s' % (i, serialNumber))
            ups_conf.write('[%d]\n' % i)
            ups_conf.write('    driver = usbhid-ups\n')
            ups_conf.write('    serial = %s\n' % serialNumber)
            ups_conf.write('    port = auto\n')
    for i, serialNumber in enumerate(serialNumbers):
        cmd = '/sbin/upsdrvctl start %d' % i
        subprocess_check(cmd, verbose=True)
    time.sleep(1)
    subprocess_check('/sbin/upsd', verbose=True)
    time.sleep(5)


In [ ]:
serialNumbers = None

def restartNutIfNeeded():
    global serialNumbers
    newSerialNumbers = findSerialNumbers()
    if newSerialNumbers != serialNumbers:
        print('Serial numbers changed, restarting Nut')
        serialNumbers = newSerialNumbers
        restartNut(serialNumbers)



In [ ]:
def upsc(id):
    ret = {}
    for line in subprocess_check('upsc %s' % id).split('\n'):
        tokens = line.split(':')
        if len(tokens) == 2:
            key = tokens[0].strip()
            val = tokens[1].strip()
            try:
                val = float(val)
            except:
                pass
            ret[key] = val
    return ret


def checkUps(id, serialNumber, config):
    stat = upsc(id)
    upsLoad = stat['ups.load']
    upsMaxPower = stat['ups.realpower.nominal']
    upsPower = upsLoad / 100.0 * upsMaxPower
    errors = []
    maxLoad = 90 # percent
    if stat['input.voltage'] <= stat['input.transfer.low']:
        errors.append('Power out or input voltage low')
    if upsLoad > maxLoad:
        errors.append('High load %d%% (test >%d%%)' % (upsLoad, maxLoad))

    name = serialNumber
    machines = ''
    
    details = 'In: %dV, Batt charge %d%%, S/N %s' % (
        stat['input.voltage'], stat['battery.charge'], serialNumber)
    
    try:
        name = config.loc[serialNumber, 'Name'].strip()
        details += ', Powering %s' % config.loc[serialNumber, 'Machines'].strip()
        details += '.  <a href="%s">config</a>' % googleSheetUrl
    except:
        details +='.  Please put S/N in <a href="%s">Google sheet<a>' % googleSheetUrl
        
    details +='  (reporter %s)' % socket.gethostname()
    
    shortStatus = '%s %dV&nbsp;%dW %d%%' % (name, stat['input.voltage'], upsPower, upsLoad)

    loadMsg = 'supplying %dW (%d%% of %dW max)' % (upsPower, upsLoad, upsMaxPower)
    if errors:
        Stat.down('; '.join(errors + [loadMsg]) , details, host=name, shortname=shortStatus)
    else:
        Stat.up('Working normally, ' + loadMsg, details, host=name, shortname=shortStatus)

def sheetUrl2CsvUrl(sheetUrl):
    tokens = sheetUrl.split('/')
    assert(len(tokens) == 7)
    assert(tokens[4] == 'd')
    docHash = tokens[5]
    assert(len(docHash) > 20)
    edit = tokens[6]
    assert edit[0:9] == 'edit#gid='
    gid = edit[9:]
    return 'https://docs.google.com/spreadsheets/d/' + docHash + '/export?format=csv&gid=' + gid

In [ ]:
while True:
    restartNutIfNeeded()

    sheetContent = requests.get(sheetUrl2CsvUrl(googleSheetUrl)).content
    config = pd.read_csv(io.BytesIO(sheetContent), na_filter=False, index_col='SerialNumber')
    
    for id, serialNumber in enumerate(serialNumbers):
        checkUps(id, serialNumber, config)
        
    sleep_until_next_period(60)